In [137]:
# Importing packages

import pandas as pd
import numpy as np
import plotnine as p9

In [138]:
# Setting routes for reading in csv data

data_masterplan_input_target = 'test_mp_data.csv'
phasing_logic_input_target = 'phasing_logic_matrix.csv'

# Reading in masterplan data and phasing logic matrix

data_masterplan_input =  pd.read_csv(data_masterplan_input_target)
phasing_logic_input = pd.read_csv(phasing_logic_input_target).rename({'delivery_status':'Phasing_Note'},axis = 1)


In [117]:
# Dropnas from the project ID column from the main data_masterplan_input data frame

data_masterplan_input = data_masterplan_input.dropna(subset = ['ProjectID'],axis = 0).reset_index(drop = True)

In [118]:
# Find columns with a '%' sign and the column immediately to the left in the main masterplan input dataframe

columns_with_percent = [col for col in data_masterplan_input.columns if '%' in col]

selected_columns = []

for col in columns_with_percent:

    # Find the index of the column with a '%' sign
    
    col_index = data_masterplan_input.columns.get_loc(col)
    # Append the column to the left if it exists
    
    if col_index > 0:
        selected_columns.append(data_masterplan_input.columns[col_index - 1])
    
    # Append the column with a '%' sign
    
    selected_columns.append(col)

# Select the identified columns from the DataFrame
committed_development = data_masterplan_input[selected_columns]
committed_development = pd.concat([data_masterplan_input[['ProjectID','Phasing_Note','Approved Completion Year']],committed_development],axis = 1)

In [119]:
# Setting start dates of the plan

plan_start = 2024
plan_end = 2040

# Creating sequence containing all years of the plan phasing period

phasing_period = np.arange(plan_start,plan_end+1,1)
phasing_period = phasing_period.astype(str)
print(phasing_period)


['2024' '2025' '2026' '2027' '2028' '2029' '2030' '2031' '2032' '2033'
 '2034' '2035' '2036' '2037' '2038' '2039' '2040']


In [120]:
# Creating dataframe includes the manual phasing overrides

phasing_override_input = data_masterplan_input[phasing_period]
phasing_override_input = pd.concat([data_masterplan_input['ProjectID'],phasing_override_input],axis = 1)
phasing_override_input

,ProjectID,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Riyadh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Samha New Housing - Ph 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Samha New Housing - Ph 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABU MREIKHAH_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,GHANADHAH_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,1113.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,Al Faya Industrial - New,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,32379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# Identifying which development group committed development should be entered in based on delivery status

committed_development =committed_development.merge(phasing_logic_input,how = 'left',on = 'Phasing_Note')

# Dropping masterplans without a status or are complete

committed_development= committed_development.dropna(subset=['group'],axis =0)
committed_development = committed_development[committed_development['group'] != 'Remove'].reset_index(drop = True)


In [136]:
# Creating a data frame that contains under construction development
under_construction_development = committed_development[committed_development['group'] == 'Under Construction'].reset_index(drop = True)
under_construction_development['Approved Completion Year'] = under_construction_development['Approved Completion Year'].apply(lambda x: 1 if pd.isna(x) else x).astype(int)
# Removing un-needed columns
under_construction_development = under_construction_development.drop(['group','delay_period_yrs','Phasing_Note'],axis = 1)

# Melting the data frame into long form
under_construction_development = under_construction_development.melt(id_vars= ['ProjectID','Approved Completion Year'])

# Splitting the long-form data frame into two parts and merging those parts together - the purpose being to split the % committed and number of units associated with each land use into separatre columns
under_construction_development_split_1 = under_construction_development[~under_construction_development['variable'].str.contains('%')].reset_index(drop = True).rename({'variable':'land_use','value':'number_units'},axis = 1)
under_construction_development_split_2 = under_construction_development[under_construction_development['variable'].str.contains('%')].reset_index(drop = True).drop(['ProjectID','variable','Approved Completion Year'],axis = 1).rename({'value':'per_committed'},axis = 1)
under_construction_development = pd.concat([under_construction_development_split_1,under_construction_development_split_2],axis = 1)

# Creating starting and end of phasing year columns

under_construction_development['start_year'] = plan_start
under_construction_development['end_year'] = under_construction_development['Approved Completion Year'].apply(lambda x: (plan_start + 1) if pd.isna(x) or x <= plan_start else x)

# Calculating the percentage of development over the course of each phasing period

under_construction_development['development_pa'] = under_construction_development['per_committed']/(under_construction_development['end_year'] - under_construction_development['start_year'])



In [140]:
under_construction_development

,ProjectID,Approved Completion Year,land_use,number_units,per_committed,start_year,end_year,development_pa
0,Riyadh,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
1,1099.0,1,Residential - Villas Emiratis (Units),0,0.0,2024,2025,0.0
2,2100.0,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
3,1170.1,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
4,5000.0,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
...,...,...,...,...,...,...,...,...
691,1007.0,1,Other (GFA),0,1.0,2024,2025,1.0
692,1007.38.3,2025,Other (GFA),0,1.0,2024,2025,1.0
693,23168.0,2022,Other (GFA),0,0.0,2024,2025,0.0
694,1113.6,2027,Other (GFA),0,1.0,2024,2027,0.333333


In [135]:
under_construction_development

,ProjectID,Approved Completion Year,land_use,number_units,per_committed,start_year,end_year,development_pa
0,Riyadh,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
1,1099.0,1,Residential - Villas Emiratis (Units),0,0.0,2024,2025,0.0
2,2100.0,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
3,1170.1,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
4,5000.0,1,Residential - Villas Emiratis (Units),0,1.0,2024,2025,1.0
...,...,...,...,...,...,...,...,...
691,1007.0,1,Other (GFA),0,1.0,2024,2025,1.0
692,1007.38.3,2025,Other (GFA),0,1.0,2024,2025,1.0
693,23168.0,2022,Other (GFA),0,0.0,2024,2025,0.0
694,1113.6,2027,Other (GFA),0,1.0,2024,2027,0.333333
